# 2. Merge data for analysis 
Section 1. 
intersect 





I made an environment on the cluster called soy_data_setup 
programs 
bcftools
vcftools
see conda list -n soy_data_setup
and look in workflow/envs for the soy_data_setup.yaml

Here im going to generate the files needed for the analysis by intersecting not imputing.


In [ ]:
# Data:
soysnp50k_wm82.a2_41317.vcf.gz 
cca_sbpa.vcf.gz # Wgs data as filtered in the steps before as indicated in the file name step1_wgs_filter.


In [ ]:
bcftools index soysnp50k_wm82.a2_41317.vcf.gz

bcftools view -S founders.txt soysnp50k_wm82.a2_41317.vcf.gz -Oz -o founders.vcf.gz

bcftools index founders.vcf.gz

I need the chromosomes to be called the same in both files

In [ ]:
#rename chromosomes
bcftools annotate --rename-chrs chr_names.txt founders.vcf.gz -Oz -o founders_gm.vcf.gz

#index
bcftools index founders_gm.vcf.gz

#only include chromosomes
bcftools view founders_gm.vcf.gz --regions Gm01,Gm02,Gm03,Gm04,Gm05,Gm06,Gm07,Gm08,Gm09,Gm10,Gm11,Gm12,Gm13,Gm14,Gm15,Gm16,Gm17,Gm18,Gm19,Gm20 -Oz -o founders_Gm_only.vcf.gz

bcftools index founders_Gm_only.vcf.gz

#check
bcftools query -l founders_Gm_only.vcf.gz
bcftools query -f '%CHROM\n' founders_Gm_only.vcf.gz|uniq

In [ ]:
#how meny positions SNPs are there?
bcftools query -f '%POS\n' cca_sbpa_Gm_only.vcf.gz | wc -l
#9677024
founders_gm.vcf.gz
#42195

Do the same with the bigger file with a script.
Try to submit as job: /workflow/scripts/filter_incl_chr.sh
sbatch filter_incl_chr.sh

output is cca_sbpa_Gm_only.vcf.gz


Stats:

I want to get some stats on my files at this point.

file with snps: 

bcftools stats cca_sbpa_Gm_only.vcf.gz > stats_cca.vchk

plot-vcfstats file.vchk -p plots/



Now i will intersect the two files so as to merge them and only retain the 50k SNP sequence. 

In [ ]:
#see intersect.sh in scripts directory
#then intersect_merge.sh

In [ ]:
#when the intersect and merge is done i can check how many snps i have 
bcftools view --no-header -G -m 2 -M 2 --types snps intersect.vcf.gz | wc -l
#36515
#should be around 40000! check!

#and that i have 10+572 samples
bcftools query -l intersect.vcf.gz | wc -l
#582! 
#check!

go to R!.... for the PCA!
insert R script?

come back to remove accession with missing data
and remove missingdata over 5%
and change chr names 

Look for accessions with too much missing data:
(bcftools query -f '[%GT\t]\n' intersect.vcf.gz | awk -v OFS="\t" '{for (i=1;i<=NF;i++) if ($i == "./.") sum[i]+=1 } END {for (i in sum) print i, sum[i] / NR }' | sort -k1,1n | cut -f 2)

I have one accession (064A) that has 31.50% missing data
the rest have 2.62% missing data or under
See resources/generated_data/accessions_missing_data.xlsx

In [ ]:
# filtering 
# remove missing data 5% snps and rename chromosomes to numbers for R
# remove 199A, 334A, 574A due to missing metadata like MG
# Remove accessions with missing data > 5% (0.05) that is one sample: accession 064A
# /Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/generated_data/intersect.vcf.gz
# /Users/josephinelovemachine/soyadapt_data_analysis/resources/generated_data/meta_data/Gm01_01.txt

bcftools filter -e "F_MISSING>0.05" /Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/generated_data/intersect.vcf.gz -Ou | bcftools annotate --rename-chrs /Users/josephinelovemachine/soyadapt_data_analysis/resources/generated_data/meta_data/Gm01_1.txt -Ou | bcftools view -S ^/Users/josephinelovemachine/soyadapt_data_analysis/resources/start_data/metadata/remove_accessions.txt -Oz -o intersect_filtered.vcf.gz

bcftools index intersect_filtered.vcf.gz

#before filtering
bcftools view --no-header -G -m 2 -M 2 --types snps intersect.vcf.gz | wc -l
36515
#after filtering
bcftools view --no-header -G -m 2 -M 2 --types snps intersect_filtered.vcf.gz | wc -l
35486
bcftools query -l intersect_filtered.vcf.gz | wc -l
#should be 574 and is

# new intersect

Here along the way via pedegree info i found more potential founders. 

i also found many interesting swedish accessions in grin global and so maybe part of the snp data so i will make a new intersect 

In [ ]:
# Data:
# zip and index 
bgzip -c soysnp50k_wm82.a2_41317.vcf > soysnp50k_wm82.a2_41317.vcf.gz
bcftools index soysnp50k_wm82.a2_41317.vcf.gz
bcftools index wgs_maf5.vcf.gz

snp50k=/Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/start_data/soysnp50k_wm82.a2_41317.vcf.gz 
wgs_maf5=/Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/start_data/wgs_maf5.vcf.gz

# what have i got?
bcftools query -f '%CHROM\n' wgs_maf5.vcf.gz | uniq
bcftools query -f '%CHROM\n' soysnp50k_wm82.a2_41317.vcf.gz | uniq

# Chr1...Chr20
# 1...20
# i need to change the names of the chromsomes in my snp file to match the wgs file

#change chromsome names in 50ksnp file
bcftools annotate --rename-chrs chr.txt soysnp50k_wm82.a2_41317.vcf.gz -Oz -o soysnp50k_wm82.a2.vcf.gz

bcftools index soysnp50k_wm82.a2.vcf.gz

bcftools query -f '%CHROM\n' soysnp50k_wm82.a2.vcf.gz | uniq


#extract is accession are in the 50ksnp file that i will use in my intersect

bcftools view -S intersect_accessions.txt soysnp50k_wm82.a2.vcf.gz --force-samples -Oz -o intersect_accessions.vcf.gz
bcftools index intersect_accessions.vcf.gz

In [ ]:
# check
bcftools query -f '%CHROM\n' intersect_accessions.vcf.gz | uniq
bcftools query -l intersect_accessions.vcf.gz

In [ ]:
#intersect the two files
bcftools isec wgs_maf5.vcf.gz intersect_accessions.vcf.gz -p output_dir

#merge

#the second file generated called 0002.vcf.gz contains the SNPs that are present in the first input file but not in the second input file.

#the third file generated called 0003.vcf.gz contains the SNPs that are present in the second input file but not in the first input file.

#to get the intersection of the two files we need to merge the second and third file. The --merge all option will merge the two files and keep only the SNPs that are present in both files.
cd output_dir

bgzip < 0002.vcf > 0002.vcf.gz
bgzip < 0003.vcf > 0003.vcf.gz
bcftools index 0002.vcf.gz
bcftools index 0003.vcf.gz

bcftools merge --merge all 0002.vcf.gz 0003.vcf.gz -Oz -o intersect.vcf.gz
# check 
bcftools query -f '%CHROM\n' intersect.vcf.gz | uniq

rm 0000.vcf
rm 0001.vcf
rm 0002.vcf
rm 0003.vcf


bcftools index /home/joco/faststorage/data_analysis_soy/output_dir/intersect2.vcf.gz

In [ ]:
#when the intersect and merge is done i can check how many snps i have 
bcftools view --no-header -G -m 2 -M 2 --types snps intersect2.vcf.gz | wc -l
#36515
#should be around 40000! check!

#and that i have 10+572 samples
bcftools query -l intersect2.vcf.gz | wc -l
#582! 
#check!